In [1]:
from sklearn.utils import shuffle
from pyvi import ViTokenizer #For split vietnamese words
import pandas as pd #For reading xlsx file
from gensim.parsing.preprocessing import strip_numeric,strip_short
import re
import numpy as np
import string

In [2]:
document = []
label = []
def read_file(file1,file2):
    with open(file1, 'r', encoding = 'utf-8') as f1:
        for line in f1:
            document.append(line)
    with open(file2, 'r', encoding = 'utf-8') as f2:
        for line in f2:
            label.append(int(line))
read_file('TxtData/datacomment.txt','TxtData/datalabel.txt')
print(len(document))
print(document[:5])
print(label[:5])

2016
['dùng hơn tháng , ko nhận_diện đc khuông mặt , 1 số ứng_dụng tự_động dừng , chơi game máy lang\n', 'camera ok lắm . mấy thì giá này nói_gì nữa . ai xài thấy sao chứ mình thấy thích nha . nhất là giao_diện của máy . mình là sam fan mà . sam là thích hết\n', 'mình sài 2 sim mobi cùng lúc , nhưng lúc người khác gọi được cho mình lúc không , đã đổi máy vẫn vậy cho đến khi chỉ dùng 1 sim thì mới hết\n', 'mình mua máy cách đây 6 tháng giờ gọi video zalo nói người khác ko nghe đc , đã khôi_phục cài_đặt mà vẫn ko đc , có gì hỗ_trợ mình\n', 'thất_vọng về chất_lượng máy . lướt web chậm , giật lag . cảm_ứng chậm , lag . nói chug tầm giá này có_thể mua đc máy tốt hơn\n']
[0, 1, 0, 0, 0]


In [3]:
def raw_text_preprocess(raw):
    raw = raw.lower()
    raw = re.sub(r"http\S+", "URL_Change", raw)
    raw = re.sub(r"https\S+", "URL_Change", raw)
    raw = re.sub(r"(www\S+)|(\S+@\S+)", "URL_Change", raw)
    raw = re.sub(r"([0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9])|([0-9][0-9]/[0-9][0-9])|([0-9][0-9]/[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = re.sub(r"([0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9])|([0-9][0-9]-[0-9][0-9])|([0-9][0-9]-[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = re.sub(r"(ngày [0-9][0-9] tháng [0-9][0-9] năm [0-9][0-9][0-9][0-9])|(ngày [0-9][0-9] tháng [0-9][0-9])|(tháng [0-9][0-9] năm [0-9][0-9][0-9][0-9])", "DATE_Change", raw)

    raw = strip_numeric(raw)
    raw = ViTokenizer.tokenize(raw)
    raw = strip_short(raw, minsize=2)
    return raw

In [4]:
document = [raw_text_preprocess(d) for d in document]

In [5]:
def dict_word(document):
    res = {}
    words = {}
    conten = 0
    for doc in document:
            token = doc.split(" ")
            for tok in token:
                if tok in res.keys():
                    res[tok] = res[tok] + 1
                else:
                    res.update({tok : 1})
            for key, value in res.items():
                if (value >= 3):
                    words.update({key : value})
    words = sorted(words.keys())
    return words

In [6]:
def read_data(document, label, words, fileName):
    word_list = []
    N = len(document)
    for doc in document:
        token = doc.split(" ")
        word = {}
        for tok in token:
            if tok in word.keys():
                word[tok] = word[tok] + 1
            else:
                word.update({tok : 1})
        word_list.append(word)
    data = dat = np.zeros((N, len(words)), dtype = int)
    j = 0
    for word in word_list:
        dat = []
        for key in words:
            if(key in word.keys()):
                dat.append(word[key])
            else:
                dat.append(0)
        data[j, :] = np.array(np.asarray(dat))
        j = j + 1
    with open(fileName, 'w') as f:
        for i, l in enumerate(label):
            f.write(str(l))
            f.write(" ")
            for num in data[i, :]:
                f.write(str(num))
                f.write(" ")
            f.write('\n')

In [7]:
'test{}.txt'.format(12)

'test12.txt'

In [8]:
#Chia dữ liệu làm 5 đoạn, lấy đoạn thứ i để test và còn lại để train
def divide_fold(i,document):
    p=int(len(document)/5)
    document = [raw_text_preprocess(d) for d in document]
    document_test = document[(i-1)*p:i*p]
    label_test = label[(i-1)*p:i*p]
    document_train = document[:(i-1)*p+1]+document[i*p+1:]
    label_train = label[:(i-1)*p+1]+label[i*p+1:]
    return (document_test,label_test,document_train ,label_train)

In [9]:
for i in range(1, 6):
    document_test,label_test,document_train ,label_train = divide_fold(i,document)
    words = dict_word(document_train)
    read_data(document_train,label_train, words, 'TxtBinaryData/train{}.txt'.format(i))
    read_data(document_test,label_test, words, 'TxtBinaryData/test{}.txt'.format(i))
    #print(len(document_test),len(document_train))


In [ ]:
##

In [64]:
def divide_fold(document):
    p=int(len(document))
    document = [raw_text_preprocess(d) for d in document]
    document_test = document[int(0.8*p):]
    label_test = label[int(0.8*p):]
    document_train = document[:int(0.8*p)]
    label_train = label[:int(0.8*p)]
    return (document_test,label_test,document_train ,label_train)

In [63]:
document_test,label_test,document_train ,label_train = divide_fold(document)
#print(len(document_test),len(document_train))
words = dict_word(document_train)
read_data(document_train,label_train, words, 'TxtBinaryData1/train.txt')
read_data(document_test,label_test, words, 'TxtBinaryData1/test.txt')